# Tutorial: Creating a Generative QA Pipeline with PromptNode

- **Level**: Advanced
- **Time to complete**: 15 minutes
- **Nodes Used**: `InMemoryDocumentStore`, `BM25Retriever`, `PromptNode`, `PromptTemplate`
- **Goal**: After completing this tutorial, you'll have created a generative question answering search system that uses a large language model through PromptNode with the new PromptTemplate structure. 

## Overview

Learn how to build a generative question answering pipeline using the power of LLMs with PromptNode. In this tutorial, we'll use the Wikipedia pages of [Seven Wonders of the Ancient World](https://en.wikipedia.org/wiki/Wonders_of_the_World) as Documents, but you can replace them with any text you want. 

This tutorial introduces you to the new PrompTemplate structure and explains how to use the new PrompTemplate to integrate PromptNode into a pipeline. 

## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/log-level)

## Installing Haystack

To start, let's install the latest release of Haystack with `pip`:

In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]
pip install datasets>=2.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Enabling Telemetry

Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [2]:
from haystack.telemetry import tutorial_running

tutorial_running(22)

## Initializing the DocumentStore

We'll start creating our question answering system by initializing a DocumentStore. A DocumentStore stores the Documents that the question answering system uses to find answers to your questions. In this tutorial, we're using the `InMemoryDocumentStore`.

Let's initialize our DocumentStore. 

In [3]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

> `InMemoryDocumentStore` is the simplest DocumentStore to get started with. It requires no external dependencies and it's a good option for smaller projects and debugging. But it doesn't scale up so well to larger Document collections, so it's not a good choice for production systems. To learn more about the DocumentStore and the different types of external databases that we support, see [DocumentStore](https://docs.haystack.deepset.ai/docs/document_store).

The DocumentStore is now ready. Now it's time to fill it with some Documents.

## Fetching and Writing Documents

We'll use the Wikipedia pages of [Seven Wonders of the Ancient World](https://en.wikipedia.org/wiki/Wonders_of_the_World) as Documents. We preprocessed the data and uploaded to a Hugging Face Space: [Seven Wonders](https://huggingface.co/datasets/bilgeyucel/seven-wonders). Thus, we don't need to perform any additional cleaning or splitting. 

Let's fetch the data and write it to the DocumentStore: 

In [4]:
from datasets import load_dataset

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")

document_store.write_documents(dataset)

Updating BM25 representation...:   0%|          | 0/151 [00:00<?, ? docs/s]

## Initializing the Retriever

Let's initialize a BM25Retriever and make it use the InMemoryDocumentStore we initialized earlier in this tutorial:

In [13]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store, top_k=2)

## Initializing the PromptNode

Let's define a custom prompt for PromptTemplate to use with PromptNode. As parameters, this prompt will accept Documents that our Retriever fetched from our DocumentStore and `query` we pass at runtime. To join the content of the Documents, we'll use `join()` function. To learn about using functions in PromptTemplate, check out [PromptTemplate Structure](https://docs.haystack.deepset.ai/docs/prompt_node#prompttemplate-structure). 

We'll initialize PromptNode with the PromptTemplate and `google/flan-t5-large` model. 

In [6]:
from haystack.nodes import PromptNode, PromptTemplate

lfqa_prompt = PromptTemplate(
    name="lfqa",
    prompt_text="""Synthesize a comprehensive answer from the following text for the given question. 
                             Provide a clear and concise response that summarizes the key points and information presented in the text. 
                             Your answer should be in your own words and be no longer than 50 words. 
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
)

prompt_node = PromptNode(model_name_or_path="google/flan-t5-large", default_prompt_template=lfqa_prompt)

/usr/local/lib/python3.9/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


>To learn about how to use custom templates with PromptNode, check out [Customizing PromptNode for NLP Tasks](https://haystack.deepset.ai/tutorials/21_customizing_promptnode) tutorial. 

## Defining the Pipeline

We'll use a custom pipeline with the Retriever, and PromptNode.

In [14]:
from haystack.pipelines import Pipeline

pipe = Pipeline()
pipe.add_node(component=retriever, name="retriever", inputs=["Query"])
pipe.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])

That's it! The pipeline's ready to generate answers to questions!

## Asking a Question

We use the pipeline `run()` method to ask a question.

In [23]:
output = pipe.run(query="How does Rhodes Statue look like?")

print(output["results"])

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


['The statue is a large, erecting statue of a god.']


Here are some other example queries to test:

In [11]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "How does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
]

🎉 Congratulations! You've learned how to create a generative QA system for your documents with PromptNode.